In [ ]:
import sys
import os
import random
import string
import shlex # Import shlex for shell argument quoting
import time # Import time for sleep
import tempfile # For creating temporary directories
import shutil # For robust directory removal

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import ipywidgets as widgets
from IPython.display import display, clear_output
import requests # Import requests for IP functionality

import stem.process
import stem.control # For Controller and Signal

# Global variables for Tor process and directory
tor_process = None
tor_data_dir = None
control_password = None

# --- 1. WebDriver Setup ---

# Install google-chrome-stable if not already installed or detected
if not os.path.exists('/usr/bin/google-chrome'):
    print("Installing Google Chrome stable...")
    !wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | sudo gpg --dearmor -o /usr/share/keyrings/google-chrome.gpg
    !echo "deb [arch=amd64 signed-by=/usr/share/keyrings/google-chrome.gpg] http://dl.google.com/linux/chrome/deb/ stable main" | sudo tee /etc/apt/sources.list.d/google-chrome.list > /dev/null
    !sudo apt-get update > /dev/null
    !sudo apt-get install google-chrome-stable -y > /dev/null
    print("Google Chrome stable installed.")

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev_shm_usage')

# Explicitly set the binary location for consistent behavior in Linux environments
chrome_options.binary_location = '/usr/bin/google-chrome'

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
print("Chrome WebDriver installed and initialized successfully in headless mode.")

# --- 2. URL Input and Navigation ---

url_input = widgets.Text(
    value='https://faucetearner.org/?r=508070998956',
    placeholder='Type a URL',
    description='Enter URL:',
    disabled=False
)

def navigate_to_url(url):
    # Removed clear_output(wait=True) to prevent widgets from disappearing
    print(f"Navigating to: {url}")
    driver.get(url)

navigate_button = widgets.Button(description='Go to URL')
def on_navigate_button_click(b):
    navigate_to_url(url_input.value)
navigate_button.on_click(on_navigate_button_click)

# --- 3. Click 'Get Started' Button ---

def click_get_started_button():
    try:
        get_started_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.hero-btn.primary'))
        )
        get_started_button.click()
        print("Successfully clicked the 'Get Started' button.")
    except Exception as e:
        print(f"Error clicking 'Get Started' button: {e}")

get_started_widget = widgets.Button(description='Click Get Started')
def on_get_started_button_click(b):
    click_get_started_button()
get_started_widget.on_click(on_get_started_button_click)

# --- 4. Registration Form Inputs and Auto-Generation ---

def generate_random_string(length=10, chars=string.ascii_lowercase + string.digits):
    return ''.join(random.choice(chars) for i in range(length))

username_input = widgets.Text(
    value='',
    placeholder='Enter Username',
    description='Username:',
    disabled=False
)
email_input = widgets.Text(
    value='',
    placeholder='Enter Email (e.g., example@gmail.com)',
    description='Email:',
    disabled=False
)
password_input = widgets.Text(
    value='',
    placeholder='Enter Password',
    description='Password:',
    disabled=False
)
confirm_password_input = widgets.Text(
    value='',
    placeholder='Confirm Password',
    description='Confirm Password:',
    disabled=False
)

def generate_username(b):
    username_input.value = generate_random_string(length=8)
def generate_email(b):
    email_prefix = generate_random_string(length=8, chars=string.ascii_lowercase + string.digits)
    email_input.value = f"{email_prefix}@gmail.com"
def generate_password(b):
    all_chars = string.ascii_letters + string.digits + string.punctuation
    password_input.value = generate_random_string(length=12, chars=all_chars)
def confirm_password(b):
    confirm_password_input.value = password_input.value

gen_username_button = widgets.Button(description='Generate Username')
gen_email_button = widgets.Button(description='Generate Email')
gen_password_button = widgets.Button(description='Generate Password')
gen_confirm_password_button = widgets.Button(description='Confirm Password')

gen_username_button.on_click(generate_username)
gen_email_button.on_click(generate_email)
gen_password_button.on_click(generate_password)
gen_confirm_password_button.on_click(confirm_password)

# --- 5. Fill Registration Fields ---

def fill_registration_fields(username, email, password, confirm_password):
    try:
        username_field = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'username'))
        )
        username_field.send_keys(username)

        email_field = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'email'))
        )
        email_field.send_keys(email)

        password_field = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'password'))
        )
        password_field.send_keys(password)

        confirm_password_field = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'confirmPassword'))
        )
        confirm_password_field.send_keys(confirm_password)

        print("Successfully filled registration fields using By.ID.")
    except Exception as e:
        print(f"Error filling registration fields: {e}")

fill_form_button = widgets.Button(description='Fill Registration Fields (by ID)')
def on_fill_form_button_click(b):
    current_username = username_input.value
    current_email = email_input.value
    current_password = password_input.value
    current_confirm_password = confirm_password_input.value
    fill_registration_fields(current_username, current_email, current_password, current_confirm_password)
fill_form_button.on_click(on_fill_form_button_click)

# --- 6. Click Checkbox ---

def click_checkbox():
    try:
        checkbox = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[@type='checkbox']"))
        )
        # Strictly use JavaScript click for cleaner output
        driver.execute_script("arguments[0].click();", checkbox)
        print("Successfully clicked the 'Terms and Conditions' checkbox via JavaScript.")
    except Exception as e:
        print(f"Error clicking checkbox: {e}")

click_checkbox_button = widgets.Button(description='Click Terms Checkbox')
def on_click_checkbox_button(b):
    click_checkbox()
click_checkbox_button.on_click(on_click_checkbox_button)

# --- 7. Submit Registration Form ---

def submit_registration_form():
    try:
        submit_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'registerButton'))
        )
        try:
            submit_button.click()
            print("Successfully clicked the 'Submit Registration' button via Selenium.")
        except Exception as e_click:
            print(f"Selenium click failed, attempting JavaScript click: {e_click}")
            driver.execute_script("arguments[0].click();", submit_button)
            print("Successfully clicked the 'Submit Registration' button via JavaScript.")
    except Exception as e:
        print(f"Error submitting registration form: {e}")

submit_form_button = widgets.Button(description='Submit Registration')
def on_submit_form_button_click(b):
    submit_registration_form()
submit_form_button.on_click(on_submit_form_button_click)

# --- 8. Capture Registration Output ---

def capture_output():
    try:
        # Removed clear_output(wait=True) to prevent widgets from disappearing
        print("Attempting to capture registration output...")

        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, 'body'))
        )

        current_url = driver.current_url
        print(f"Current URL after submission: {current_url}")

        try:
            success_message_element = driver.find_element(By.CSS_SELECTOR, '.alert.alert-success, .success-message, #successMessage')
            print(f"Success Message: {success_message_element.text}")
        except:
            pass

        try:
            error_message_element = driver.find_element(By.CSS_SELECTOR, '.alert.alert-danger, .error-message, #errorMessage')
            print(f"Error Message: {error_message_element.text}")
        except:
            pass

        if 'success_message_element' not in locals() and 'error_message_element' not in locals():
            page_title = driver.title
            print(f"Page Title: {page_title}")
            print("No specific success/error messages found. Please inspect the page manually if needed.")

    except Exception as e:
        print(f"Error capturing registration output: {e}")

capture_result_button = widgets.Button(description='Capture Registration Result')
def on_capture_result_button_click(b):
    capture_output()
capture_result_button.on_click(on_capture_result_button_click)

# --- 9. IP Display Functionality ---

def get_public_ip():
    try:
        response = requests.get('https://api.ipify.org')
        response.raise_for_status()
        ip_address = response.text.strip()
        print(f"Current public IP address: {ip_address}")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching public IP address: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

ip_button = widgets.Button(description='Get Public IP')
def on_ip_button_click(b):
    get_public_ip()
ip_button.on_click(on_ip_button_click)

# --- 10. Proxy Configuration ---

proxy_ip_input = widgets.Text(
    value='',
    placeholder='e.g., 192.168.1.1',
    description='Proxy IP:',
    disabled=False
)

proxy_port_input = widgets.Text(
    value='',
    placeholder='e.g., 8080',
    description='Proxy Port:',
    disabled=False
)

def configure_driver_with_proxy(proxy_ip=None, proxy_port=None):
    global driver, service
    print(f"Attempting to re-initialize WebDriver. Current driver object: {driver}")

    if 'driver' in globals() and driver is not None:
        try:
            driver.quit()
            print("Previous WebDriver instance quit successfully.")
        except Exception as e:
            print(f"Error quitting previous WebDriver instance: {e}")

    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev_shm_usage')

    if proxy_ip and proxy_port:
        proxy_address = f'{proxy_ip}:{proxy_port}'
        chrome_options.add_argument(f'--proxy-server={proxy_address}')
        print(f"Configuring WebDriver with proxy: {proxy_address}")
    else:
        print("Configuring WebDriver without a proxy.")

    # Explicitly set the binary location for consistent behavior in Linux environments
    chrome_options.binary_location = '/usr/bin/google-chrome'

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        print("New Chrome WebDriver initialized successfully.")
    except Exception as e:
        print(f"Error initializing Chrome WebDriver: {e}")
        driver = None

apply_proxy_button = widgets.Button(description='Apply Proxy / Reconfigure WebDriver')
def on_apply_proxy_button_click(b):
    current_proxy_ip = proxy_ip_input.value.strip()
    current_proxy_port = proxy_port_input.value.strip()

    if current_proxy_ip and not current_proxy_port:
        print("Please enter a proxy port if a proxy IP is provided.")
        return

    if not current_proxy_ip and not current_proxy_port:
        configure_driver_with_proxy(None, None)
    else:
        configure_driver_with_proxy(current_proxy_ip, current_proxy_port)
apply_proxy_button.on_click(on_apply_proxy_button_click)

gen_proxy_ip_button = widgets.Button(description='Generate Proxy IP')
def on_gen_proxy_ip_button_click(b):
    # This is a placeholder; real proxy generation is complex
    proxy_ip_input.value = '1.2.3.4' # Example IP
    print("Generated a placeholder proxy IP. Please replace with a real one.")
gen_proxy_ip_button.on_click(on_gen_proxy_ip_button_click)

gen_proxy_port_button = widgets.Button(description='Generate Proxy Port')
def on_gen_proxy_port_button_click(b):
    # This is a placeholder; real proxy generation is complex
    proxy_port_input.value = '8080' # Example Port
    print("Generated a placeholder proxy port. Please replace with a real one.")
gen_proxy_port_button.on_click(on_gen_proxy_port_button_click)

# --- 11. Tor Integration ---

# Install Tor daemon if not already installed (ensures apt-get update happens before trying to install tor)
if not os.path.exists('/usr/bin/tor'): # Check if Tor is already installed
    print("Updating package list and installing Tor daemon...")
    !sudo apt-get update -y > /dev/null
    !sudo apt-get install tor -y > /dev/null
    print("Tor daemon installed successfully.")

# Cleanup function for Tor process and temporary directory
def cleanup_tor():
    global tor_process, tor_data_dir
    if tor_process:
        try:
            print("Stopping Tor daemon...")
            tor_process.kill() # Terminate the Tor process
            tor_process = None
            print("Tor daemon stopped.")
        except Exception as e:
            print(f"Error stopping Tor daemon: {e}")
    if tor_data_dir and os.path.exists(tor_data_dir):
        try:
            shutil.rmtree(tor_data_dir)
            tor_data_dir = None
            print(f"Removed temporary Tor data directory.")
        except Exception as e:
            print(f"Error removing temporary Tor data directory: {e}")

# Function to generate a random password
def generate_random_password(length=16):
    chars = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(chars) for i in range(length))

# Call cleanup to remove previous temporary data and stop known processes
cleanup_tor()

# Forcefully kill any lingering Tor processes to ensure ports are free
print("Attempting to kill any lingering Tor processes...")
!pkill -f tor || true # '|| true' prevents error if no process is found
print("Lingering Tor processes killed (if any).")

# Generate a random password for the control port
global control_password
control_password = generate_random_password()

# Use a shell command to get the hashed password from the tor binary
print(f"Generating hashed password using tor binary...")
quoted_password = shlex.quote(control_password)
hashed_password_output = !tor --hash-password {quoted_password}

hashed_password = ""
for line in hashed_password_output:
    if line.strip().startswith("16:"):
        hashed_password = line.strip()
        break

if not hashed_password:
    print("Error: Could not extract hashed password from tor output.")
    print(f"Tor output: {hashed_password_output}")
else:
    print(f"Hashed password generated: {hashed_password}")

# Create a temporary directory for Tor's data, which is required
tor_data_dir = tempfile.mkdtemp()

print("Starting Tor daemon...")
try:
    # Start the Tor daemon in the background with an increased timeout
    tor_process = stem.process.launch_tor_with_config(
        config={
            'SocksPort': '9050',
            'ControlPort': '9051',
            'HashedControlPassword': hashed_password,
            'DataDirectory': tor_data_dir
        },
        init_msg_handler=lambda line: print(f"Tor init: {line}"), # Optional: print Tor's initialization messages
        timeout=300 # Increased timeout to 5 minutes (300 seconds)
    )
    print(f"Tor daemon started successfully. SOCKS proxy on localhost:9050, Control port on localhost:9051.")
    print(f"Tor Control Password (use this if programmatic control needs explicit auth): {control_password}")
    time.sleep(10) # Give Tor a moment to fully stabilize after bootstrapping
except Exception as e:
    print(f"Error starting Tor daemon: {e}")
    cleanup_tor()


def configure_driver_with_tor_proxy():
    global driver, service
    print("Attempting to re-initialize WebDriver for Tor proxy...")

    if 'driver' in globals() and driver is not None:
        try:
            driver.quit()
            print("Previous WebDriver instance quit successfully.")
        except Exception as e:
            print(f"Error quitting previous WebDriver instance: {e}")

    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev_shm_usage')

    # Configure Chrome to use Tor's SOCKS5 proxy
    chrome_options.add_argument('--proxy-server=socks5://127.0.0.1:9050')
    print("Configuring WebDriver with Tor SOCKS5 proxy: 127.0.0.1:9050")

    # Explicitly set the binary location for consistent behavior in Linux environments
    chrome_options.binary_location = '/usr/bin/google-chrome'

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        print("New Chrome WebDriver initialized successfully with Tor proxy.")
    except Exception as e:
        print(f"Error initializing Chrome WebDriver with Tor proxy: {e}")
        driver = None

apply_tor_proxy_button = widgets.Button(description='Apply Tor Proxy / Reconfigure WebDriver')
def on_apply_tor_proxy_button_click(b):
    configure_driver_with_tor_proxy()
apply_tor_proxy_button.on_click(on_apply_tor_proxy_button_click)

def renew_tor_circuit():
    global control_password # Ensure control_password is accessible
    try:
        # Connect to the Tor controller
        with stem.control.Controller.from_port(port=9051) as controller:
            # Authenticate using the control password generated earlier
            controller.authenticate(password=control_password)

            # Send the NEWNYM signal to request a new circuit (new IP)
            controller.signal(stem.Signal.NEWNYM)
            print("Successfully requested a new Tor circuit (new IP).")
            print("It may take a few seconds for the new IP to be fully active.")

    except Exception as e:
        print(f"Error renewing Tor circuit: {e}")

renew_tor_ip_button = widgets.Button(description='Renew Tor IP')
def on_renew_tor_ip_button_click(b):
    renew_tor_circuit()
renew_tor_ip_button.on_click(on_renew_tor_ip_button_click)

# --- 12. Automated Loop Functionality ---

loop_count_input = widgets.IntText(
    value=1,
    description='Number of Loops:',
    min=1,
    disabled=False
)

def automate_all_steps_in_loop(num_loops):
    for i in range(num_loops):
        print(f"\n--- Starting automation loop {i+1}/{num_loops} ---")

        # 1. Apply Tor Proxy
        print("Step 1: Applying Tor Proxy...")
        configure_driver_with_tor_proxy()
        time.sleep(5) # Give driver a moment to initialize and proxy to set up

        # 2. Go to URL
        print("Step 2: Navigating to URL...")
        navigate_to_url(url_input.value)
        time.sleep(3) # Give page a moment to load

        # 3. Click 'Get Started'
        print("Step 3: Clicking 'Get Started' button...")
        click_get_started_button()
        time.sleep(3) # Give page a moment to load after click

        # 4. Generate & Fill Registration Fields (all in sequence)
        print("Step 4: Generating username...")
        generate_username(None)
        time.sleep(0.5)
        print("Step 4: Generating email...")
        generate_email(None)
        time.sleep(0.5)
        print("Step 4: Generating password...")
        generate_password(None)
        time.sleep(0.5)
        print("Step 4: Confirming password...")
        confirm_password(None)
        time.sleep(0.5)
        print("Step 4: Filling registration fields...")
        fill_registration_fields(username_input.value, email_input.value, password_input.value, confirm_password_input.value)
        time.sleep(3)

        # 5. Click Terms Checkbox
        print("Step 5: Clicking Terms Checkbox...")
        click_checkbox()
        time.sleep(3)

        # 6. Submit Registration
        print("Step 6: Submitting Registration Form...")
        submit_registration_form()
        time.sleep(5) # Longer wait for form submission processing

        # 7. Capture Registration Output
        print("Step 7: Capturing Registration Result...")
        capture_output()
        time.sleep(2)

        # 8. Get public IP after each loop to confirm Tor changes
        print("Step 8: Getting current public IP...")
        get_public_ip()
        time.sleep(2)

        print(f"--- Finished automation loop {i+1}/{num_loops} ---")

automate_button = widgets.Button(description='Automate All Steps (Looped)')
def on_automate_button_click(b):
    clear_output(wait=True) # Clear output once before starting the entire automation process
    automate_all_steps_in_loop(loop_count_input.value)
    display(all_widgets_display) # Re-display the widgets after the automation is done
automate_button.on_click(on_automate_button_click)

# --- 13. Display All Widgets ---

username_row = widgets.HBox([username_input, gen_username_button])
email_row = widgets.HBox([email_input, gen_email_button])
password_row = widgets.HBox([password_input, gen_password_button])
confirm_password_row = widgets.HBox([confirm_password_input, gen_confirm_password_button])

proxy_ip_row = widgets.HBox([proxy_ip_input, gen_proxy_ip_button])
proxy_port_row = widgets.HBox([proxy_port_input, gen_proxy_port_button])

all_widgets_display = widgets.VBox([
    widgets.HBox([url_input, navigate_button]),
    get_started_widget,
    username_row,
    email_row,
    password_row,
    confirm_password_row,
    fill_form_button,
    click_checkbox_button,
    submit_form_button,
    capture_result_button,
    ip_button,
    proxy_ip_row,
    proxy_port_row,
    apply_proxy_button,
    apply_tor_proxy_button,
    renew_tor_ip_button,
    widgets.HBox([loop_count_input, automate_button]) # Added automation controls
])

display(all_widgets_display)
print("All interactive control widgets displayed.")
